In [6]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('../cleaned_data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1851422 entries, 0 to 1851421
Data columns (total 36 columns):
 #   Column             Dtype  
---  ------             -----  
 0   FlightDate         object 
 1   Quarter            int64  
 2   Year               int64  
 3   Month              int64  
 4   DayofMonth         int64  
 5   DepTime            float64
 6   DepDel15           float64
 7   CRSDepTime         int64  
 8   DepDelayMinutes    float64
 9   OriginAirportID    int64  
 10  DestAirportID      int64  
 11  ArrTime            float64
 12  CRSArrTime         int64  
 13  ArrDelayMinutes    float64
 14  Origin             object 
 15  Dest               object 
 16  RoundedFlightDate  object 
 17  DepatHr            int64  
 18  ArrDel15           float64
 19  Delayed            int64  
 20  date               object 
 21  airport            object 
 22  windspeedKmph      int64  
 23  winddirDegree      int64  
 24  weatherCode        int64  
 25  precipMM          

In [10]:
label_encoder = LabelEncoder()
data['Encoded_Origin_Airport'] = label_encoder.fit_transform(data['Origin'])
data['Encoded_Dest_Airport'] = label_encoder.fit_transform(data['Dest'])

In [4]:
# data = pd.concat([data, pd.get_dummies(data['Origin'],prefix='Origin')], axis=1)
# data = pd.concat([data, pd.get_dummies(data['Dest'],prefix='Destination')], axis=1)

In [12]:
# Df = data[[ 'Quarter', 'Month', 'DayofMonth',
#        'CRSDepTime', 'CRSArrTime', 
#        'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
#        'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph',
#        'tempF', 'WindChillF', 'humidity',  'Origin_ATL',
#        'Origin_CLT', 'Origin_DEN', 'Origin_DFW', 'Origin_EWR', 'Origin_IAH',
#        'Origin_JFK', 'Origin_LAS', 'Origin_LAX', 'Origin_MCO', 'Origin_MIA',
#        'Origin_ORD', 'Origin_PHX', 'Origin_SEA', 'Origin_SFO',
#        'Destination_ATL', 'Destination_CLT', 'Destination_DEN',
#        'Destination_DFW', 'Destination_EWR', 'Destination_IAH',
#        'Destination_JFK', 'Destination_LAS', 'Destination_LAX',
#        'Destination_MCO', 'Destination_MIA', 'Destination_ORD',
#        'Destination_PHX', 'Destination_SEA', 'Destination_SFO',  'DepDel15']]

Df = data[[ 'Quarter', 'Month', 'DayofMonth',
       'CRSDepTime', 'CRSArrTime', 
       'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
       'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph',
       'tempF', 'WindChillF', 'humidity', 'Encoded_Origin_Airport', 'Encoded_Dest_Airport' ,'DepDel15']]


In [13]:
Df.head()

,Quarter,Month,DayofMonth,CRSDepTime,CRSArrTime,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,Encoded_Origin_Airport,Encoded_Dest_Airport,DepDel15
0,1,1,1,2347,714,15,123,113,0.0,10,1028,0,17,26,29,21,59,7,10,1.0
1,1,1,1,20,705,15,123,113,0.0,10,1028,0,17,26,29,21,59,7,0,0.0
2,1,1,1,2359,603,15,38,113,0.0,10,1020,0,29,17,59,58,39,8,11,0.0
3,1,1,1,30,823,15,38,113,0.0,10,1020,0,29,17,59,58,39,8,10,0.0
4,1,1,1,10,501,15,38,113,0.0,10,1020,0,29,17,59,58,39,8,3,0.0


In [14]:
X = Df.iloc[:, :-1]
y = Df.iloc[:, -1]

In [15]:
## Lead to data leakage

# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler(random_state=17)
# X, y = ros.fit_resample(X, y)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)

In [17]:
X_train.shape, y_train.shape

((1388566, 19), (1388566,))

In [18]:
y_train.value_counts()

0.0    1109516
1.0     279050
Name: DepDel15, dtype: int64

In [19]:
ratio = 1109516/279050
ratio

3.976047303350654

4:1 ratio

In [20]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
ros = RandomOverSampler(random_state=42)
smote = SMOTE(random_state = 42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [21]:
y_train_smote.value_counts()

0.0    1109516
1.0    1109516
Name: DepDel15, dtype: int64

In [22]:
weights = {0:1.0, 1:4.0}
cls = DecisionTreeClassifier(random_state=42), class_weight=weights)

In [23]:
cls.fit(X_train_smote.values, y_train_smote.values)

DecisionTreeClassifier(class_weight={0: 1.0, 1: 4.0}, random_state=42)

In [121]:
X_test.values[:100]

array([[ 2.,  5., 20., ...,  0.,  0.,  0.],
       [ 4., 10.,  9., ...,  0.,  0.,  0.],
       [ 3.,  9.,  1., ...,  0.,  1.,  0.],
       ...,
       [ 2.,  5., 26., ...,  0.,  0.,  0.],
       [ 2.,  4., 17., ...,  0.,  0.,  0.],
       [ 2.,  6., 21., ...,  0.,  0.,  0.]])

In [24]:
prediction = cls.predict(X_test.values)
prediction[:100]

array([0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0.])

In [25]:
y_test.values[:100]

array([0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [26]:
print(classification_report(prediction, y_test.values))

              precision    recall  f1-score   support

         0.0       0.85      0.86      0.85    366864
         1.0       0.44      0.43      0.43     95992

    accuracy                           0.77    462856
   macro avg       0.64      0.64      0.64    462856
weighted avg       0.77      0.77      0.77    462856



In [27]:
accuracy_score(prediction, y_test.values)

0.7680596124928704

In [28]:
f1_score(prediction, y_test.values)

0.43203521376384135

In [71]:
# params = {'criterion': ['gini', 'entropy'], 'max_depth': [40,50,55,60], "max_features": [None, 'sqrt', 'log2'], 'class_weight':[{0:1,1:4}, {0:1,1:3}, {0:1,1:5}]}
# dt = DecisionTreeClassifier()
# grid = GridSearchCV(dt, param_grid=params, n_jobs=-1)

In [59]:
# grid.fit(X_train.values, y_train.values)

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'class_weight': [{0: 1, 1: 4}, {0: 1, 1: 3},
                                          {0: 1, 1: 5}],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [40, 50, 55, 60],
                         'max_features': [None, 'sqrt', 'log2']})

In [60]:
# grid.best_params_

{'class_weight': {0: 1, 1: 3},
 'criterion': 'entropy',
 'max_depth': 55,
 'max_features': None}

In [61]:
grid.best_score_

0.7659175003397709

In [65]:
# dt.set_params(**grid.best_params_)

DecisionTreeClassifier(class_weight={0: 1, 1: 3}, criterion='entropy',
                       max_depth=55)

In [67]:
# dt.fit(X_train.values, y_train.values)

DecisionTreeClassifier(class_weight={0: 1, 1: 3}, criterion='entropy',
                       max_depth=55)

In [68]:
# prediction = dt.predict(X_test.values)
# prediction[:100]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.])

In [70]:
# print(classification_report(prediction, y_test.values))

              precision    recall  f1-score   support

         0.0       0.86      0.86      0.86    373243
         1.0       0.42      0.43      0.43     89613

    accuracy                           0.77    462856
   macro avg       0.64      0.64      0.64    462856
weighted avg       0.78      0.77      0.78    462856

